In [11]:
import numpy as np

# ------------------------------------------------------------
# Dummy Environment for Demonstration
# ------------------------------------------------------------
class DummyEnv:
    """
    A simple dummy environment.
    It assumes:
    - The environment has 'num_states' discrete states.
    - There are 'num_actions' possible actions.
    - The 'step' function returns a random next state, a random reward,
      and a 10% chance to terminate the episode.
    """
    def __init__(self, num_states, num_actions):
        self.num_states = num_states
        self.num_actions = num_actions
        self.current_state = 0
        
    def set_state(self, state):
        self.current_state = state
        
    def step(self, action):
        # Randomly transition to a new state
        next_state = np.random.choice(range(self.num_states))
        # Generate a random reward (for example, normally distributed)
        reward = np.random.randn()
        # End the episode with a 10% probability
        done = np.random.rand() < 0.1
        info = {}
        return next_state, reward, done, info

# ------------------------------------------------------------
# Define State and Action Spaces
# ------------------------------------------------------------
num_states = 10      # Example: 10 discrete states
num_actions = 4      # Example: 4 possible actions
S = range(num_states)
A = range(num_actions)

# ------------------------------------------------------------
# Initialize Q-Table and Other Variables
# ------------------------------------------------------------
# Q-table: one row per state and one column per action, initialized to zeros.
Q = np.zeros((num_states, num_actions))

# Arrays to accumulate returns for each state-action pair.
returns_sum = np.zeros((num_states, num_actions))
returns_count = np.zeros((num_states, num_actions))

# ------------------------------------------------------------
# Define the Epsilon-Greedy Policy
# ------------------------------------------------------------
def epsilon_greedy_policy(state, epsilon=0.1):
    """
    Chooses a random action with probability epsilon,
    otherwise returns the action with the highest Q-value for the given state.
    """
    if np.random.rand() < epsilon:
        return np.random.choice(list(A))
    else:
        return np.argmax(Q[state])

# ------------------------------------------------------------
# Implementing Exploring Starts
# ------------------------------------------------------------
def exploring_starts():
    """
    Randomly selects a starting state and action,
    ensuring every state-action pair can be explored.
    """
    s0 = np.random.choice(list(S))
    a0 = np.random.choice(list(A))
    return s0, a0

# ------------------------------------------------------------
# Generate an Episode Using the Current Policy
# ------------------------------------------------------------
def generate_episode(env, start_state, start_action):
    """
    Generates an episode starting from a given state and action.
    Returns a list of (state, action, reward) tuples.
    """
    env.set_state(start_state)
    state = start_state
    action = start_action
    episode = []
    done = False

    while not done:
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        if not done:
            action = epsilon_greedy_policy(next_state)
        state = next_state
        
    return episode

# ------------------------------------------------------------
# Initialize the Environment
# ------------------------------------------------------------
env = DummyEnv(num_states, num_actions)

# ------------------------------------------------------------
# Monte Carlo Exploring Starts (MC ES) Main Loop
# ------------------------------------------------------------
num_episodes = 50000  # Example number of episodes

for episode_i in range(num_episodes):
    # 1. Choose a random starting state-action pair (Exploring Starts)
    s0, a0 = exploring_starts()
    
    # 2. Generate an episode using the current policy
    episode = generate_episode(env, s0, a0)
    
    # 3. Process the episode in reverse to compute returns and update Q-values.
    G = 0
    # Iterate over the episode in reverse order
    for t in reversed(range(len(episode))):
        s_t, a_t, reward = episode[t]
        G += reward
        
        # First-visit check: update only if (s_t, a_t) is not seen earlier in the episode.
        if (s_t, a_t) not in [(episode[i][0], episode[i][1]) for i in range(t)]:
            returns_sum[s_t, a_t] += G
            returns_count[s_t, a_t] += 1.0
            Q[s_t, a_t] = returns_sum[s_t, a_t] / returns_count[s_t, a_t]

    # 4. The policy is implicitly updated by the epsilon-greedy function,
    #    which always selects the action with the highest Q-value.
    
# ------------------------------------------------------------
# Final Output
# ------------------------------------------------------------
print("Final Q-Table after {} episodes:".format(num_episodes))
print(Q)


Final Q-Table after 50000 episodes:
[[-0.0492369   0.02943397 -0.00288413  0.01757525]
 [-0.10233109  0.02385264 -0.0707276  -0.14933399]
 [ 0.02906495  0.00708541 -0.03501358  0.00748026]
 [-0.07649729 -0.02783598 -0.0403287  -0.0192394 ]
 [ 0.00932047  0.01452559 -0.04980954 -0.03145595]
 [-0.03979207 -0.02422997 -0.01137551 -0.04483987]
 [-0.00688771 -0.00296477 -0.0301759  -0.00843499]
 [-0.02094065  0.01035476 -0.06280117  0.02339754]
 [-0.03745759 -0.11238694  0.00853358 -0.13587449]
 [ 0.01746955 -0.09678515 -0.0150934   0.01448133]]
